<a href="https://colab.research.google.com/github/luke-scot/emissions-tracking/blob/main/energy_consumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Energy consumption


Run the first two cells setup the notebook.





In [1]:
%%capture
"""Installation and downloads"""
# Install floweaver and display widget packages
%pip install floweaver ipysankeywidget

# Import necessary packages
from floweaver import *

"""Import example data"""
# Import packages
import gdown, os
from google.colab import files

# Import and unzip files -> You can then view them in the left files panel
folder, zip_path = 'example_data', 'example_data.zip'
if not os.path.exists(folder): 
  gdown.download('https://drive.google.com/uc?id=1qriY29v7eKJIs07UxAw5RlJirfwuLnyP', zip_path ,quiet=True)
  ! unzip $zip_path -d 'example_data'
  ! rm $zip_path

In [10]:
"""Display setup"""
# Enable widget display for Sankeys in Colab
from google.colab import output
output.enable_custom_widget_manager()

"""Download US files for as csv examples"""
download = [files.download('example_data/'+filename) for filename in ['us-energy-consumption.csv','us-energy-consumption-processes.csv']]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Task 1 - US example 

Step through this section to see an example for the US based on the [Sankey diagrams of US energy consumption from the Lawrence Livermore National Laboratory](https://flowcharts.llnl.gov/) (thanks to John Muth for the suggestion and transcribing the data)

In [11]:
"""Load the dataset"""
dataset = Dataset.from_csv('example_data/us-energy-consumption.csv',
                           dim_process_filename='example_data/us-energy-consumption-processes.csv')

In [12]:
"""Define the order the nodes appear in"""
sources = ['Solar', 'Nuclear', 'Hydro', 'Wind', 'Geothermal',
           'Natural_Gas', 'Coal', 'Biomass', 'Petroleum']

uses = ['Residential', 'Commercial', 'Industrial', 'Transportation']

In [13]:
"""define the Sankey diagram definition"""
nodes = {
    'sources': ProcessGroup('type == "source"', Partition.Simple('process', sources), title='Sources'),
    'imports': ProcessGroup(['Net_Electricity_Import'], title='Net electricity imports'),
    'electricity': ProcessGroup(['Electricity_Generation'], title='Electricity Generation'),
    'uses': ProcessGroup('type == "use"', partition=Partition.Simple('process', uses)),
    
    'energy_services': ProcessGroup(['Energy_Services'], title='Energy services'),
    'rejected': ProcessGroup(['Rejected_Energy'], title='Rejected energy'),
    
    'direct_use': Waypoint(Partition.Simple('source', [
        # This is a hack to hide the labels of the partition, there should be a better way...
        (' '*i, [k]) for i, k in enumerate(sources)
    ])),
}

ordering = [
    [[], ['sources'], []],
    [['imports'], ['electricity', 'direct_use'], []],
    [[], ['uses'], []],
    [[], ['rejected', 'energy_services'], []]
]

bundles = [
    Bundle('sources', 'electricity'),
    Bundle('sources', 'uses', waypoints=['direct_use']),
    Bundle('electricity', 'uses'),
    Bundle('imports', 'uses'),
    Bundle('uses', 'energy_services'),
    Bundle('uses', 'rejected'),
    Bundle('electricity', 'rejected'),
]

In [16]:
"""Define the colours to roughly imitate the original Sankey diagram"""
palette = {
    'Solar': 'gold',
    'Nuclear': 'red',
    'Hydro': 'blue',
    'Wind': 'purple',
    'Geothermal': 'brown',
    'Natural_Gas': 'steelblue',
    'Coal': 'black',
    'Biomass': 'lightgreen',
    'Petroleum': 'green',
    'Electricity': 'orange',
    'Rejected energy': 'lightgrey',
    'Energy services': 'dimgrey',
}

And here's the result!

In [17]:
sdd = SankeyDefinition(nodes, bundles, ordering,
                       flow_partition=dataset.partition('type'))
weave(sdd, dataset, palette=palette) \
    .to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

You can save a copy of the Sankey by adding `.auto_save_png('filename.png')` or `.auto_save_svg('filename.svg')` to the end of the `weave` call in the previous box.

## Task 2 - Create your own

Follow the steps below to create an equivalent Sankey for your own country.

  1. Find and download the IEA World Energy Balances Highlights spreadsheet, from the webpage: https://www.iea.org/reports/world-energy-balances-overview. Then upload it to Colab using the `upload` button in the left panel.

  2. Filter your country data in the Timeseries_1971-2019 tab, and choose the 2018 year.



In [27]:
"""Read in an Excel file"""
import pandas as pd
data = pd.read_excel('WorldEnergyBalancesHighlights2021.xlsx',sheet_name='TimeSeries_1971-2020',header=1)

In [28]:
data

,Country,Product,Flow,NoCountry,NoProduct,NoFlow,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020 Provisional,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64
0,Australia,"Coal, peat and oil shale",Production (PJ),01. Australia,"01. Coal, peat and oil shale",01. Production (PJ),1368.25,1648.41,1685.22,1640.36,1874.31,1848.93,2047.78,2007.46,2175.88,2172.86,2621.7,2651.69,3033.66,3305.8,3514.37,3789.07,4302.48,3771.2,4195.74,4442.32,4804.89,4988.13,5023.44,4969.2,5351.71,5441,5889.83,6381.83,6470.25,6890.56,7470.64,7697.99,7697.87,7920.97,8439.69,8533.05,9000.75,9039.73,9350.5,10323.1,9782.07,10272.4,11074.8,11950.8,12501,12223.6,12264.2,12034.6,12596.4,12331.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Australia,"Coal, peat and oil shale",Imports (PJ),01. Australia,"01. Coal, peat and oil shale",02. Imports (PJ),0,0,0,0,0,0,0,0.292657,0.366286,0.0491028,0,0.0249784,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.458,0.405002,0.593998,1.377,1.188,0.189001,0.162,1.7846,6.75564,3.63543,2.57749,5.28741,6.9788,6.82999,7.91674,12.7357,17.0379,19.6295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Australia,"Coal, peat and oil shale",Exports (PJ),01. Australia,"01. Coal, peat and oil shale",03. Exports (PJ),-541.424,-614.718,-738.939,-745.383,-849.708,-786.847,-915.089,-1001.53,-1103.71,-1164.25,-1385.94,-1268.28,-1677.59,-2081.99,-2407.11,-2497.85,-2740.53,-2721.26,-2699.04,-2816.32,-3274.68,-3433.68,-3584.96,-3563.81,-3709.27,-3823.57,-4251.28,-4524.54,-4676.71,-5083.85,-5233.9,-5549.04,-5674.78,-5957.9,-6321.66,-6312.86,-6651.13,-6864.84,-7077.71,-7971.29,-7720.39,-8171.02,-9101.06,-10166.9,-10634.4,-10552.5,-10248,-10333.4,-10629,-10541.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Australia,"Coal, peat and oil shale",Total energy supply (PJ),01. Australia,"01. Coal, peat and oil shale",04. Total energy supply (PJ),884.297,923.25,945.542,928.323,998.127,996.367,1081.56,998.11,1034.39,1143.76,1155.06,1195.93,1169.64,1226.32,1260.8,1237.15,1331.82,1347.23,1464.49,1460.68,1518.89,1542.08,1572.51,1553.85,1569.97,1703.48,1855.93,1972.87,2013.57,2015.83,2018.54,2034.81,2015.73,2089.2,2136.73,2164.53,2172.8,2156.3,2205.22,2113.08,2013.81,1968.85,1831.2,1736.61,1797.04,1863.05,1872.03,1797.65,1744.11,1648.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Australia,"Coal, peat and oil shale","Electricity, CHP and heat plants (PJ)",01. Australia,"01. Coal, peat and oil shale","05. Electricity, CHP and heat plants (PJ)",-516.539,-569.922,-577.9,-617.948,-664.427,-684.195,-770.445,-707.324,-734.531,-844.413,-870.852,-913.252,-875.941,-897.846,-987.376,-978.452,-1058.49,-1111.71,-1199.82,-1210.97,-1253.83,-1286.19,-1294.48,-1310.84,-1355.83,-1435.71,-1486.67,-1675.75,-1701.33,-1724.44,-1793.46,-1849.53,-1773.1,-1855.04,-1903.3,-1942.67,-1960.66,-1933.78,-2021.51,-1877.01,-1789.2,-1769.93,-1650.41,-1561.05,-1622.71,-1668.87,-1651.52,-1573.93,-1531.52,..,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6155,IEA and Accession/Association countries,Total,Electricity output (GWh),54. IEA and Accession/Association countries,11. Total,13. Electricity output (GWh),..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,9.1041e+06,9.38705e+06,9.56241e+06,9.88293e+06,1.02606e+07,1.06607e+07,1.10464e+07,1.12927e+07,1.15877e+07,1.19179e+07,1.24755e+07,1.25026e+07,1.30244e+07,1.34846e+07,1.41145e+07,1.47266e+07,1.52677e+07,1.59823e+07,1.62521e+07,1.61906e+07,1.72991e+07,1.78873e+07,1.82535e+07,1.8833e+07,1.92812e+07,1.94347e+07,1.99507e+07,2.05896e+07,2.13663e+07,2.16413e+07,2.16196e+07,

3. Check out the IEA Sankey as a guide from: https://www.iea.org/sankey/

  4. Organise data into the correct form for the csv files (using the two files: us-energy-consumption.csv and us-energy-consumption-processes.csv as your base - find these in your Downloads folder). This will involve allocating the IEA balance categories to the labels used in the US example. Flows into and out of each slice need to be balanced.  And the IEA has no allocation to 'energy services' and 'rejected energy' so you will need to guess, or use the ratios from the US example.  You can also check your numbers using SankeyMATIC. 

  5. Adapt the notebook to create a Sankey for your country.

  6. Save a copy by running the final cell